# Probability Visualization Notebook
## Import useful packages
Author: Lawrence Lechuga 
Date: 08/21/2020

In [1]:
import tkinter as tk
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)
import numpy as np
from matplotlib.figure import Figure
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import multivariate_normal
from scipy.stats.contingency import margins
from matplotlib import pyplot

## Define useful functions
Here we define two functions: submit() and initial_cond(). The first generates some of the GUI functionality. The secondwill initialize some parameters for our gaussian distribution, such as the mean vector, the covariance matrix, number of points, and axes limit.

In [2]:
def submit():
    """Button functionality added here. This will get the values of the text entry
    boxes and update the graphs accordingly"""

    cov = []
    mu = []
    # Get the values from the text entry boxes
    if (len(entxx.get()) !=0 and len(entxy.get())!=0 and len(entyx.get())!=0 and len(entyy.get())!=0 
       and len(entmux.get()) !=0 and len(entmuy.get()) !=0):
        
        mu.append(float(entmux.get())) 
        mu.append(float(entmuy.get()))
        cov.append(float(entxx.get()))
        cov.append(float(entxy.get()))
        cov.append(float(entyx.get()))
        cov.append(float(entyy.get()))
        # Create covariance matrix
        Sig = np.reshape(np.array(cov), (2,2))
        # create covariance matrix
        Mu = np.array(mu)
        # Recalculate new parameters
        X, Y, pos = initial_cond(Mu,Sig)
        F = multivariate_normal(Mu, Sig)
        Z = F.pdf(pos)
    else:
        cov.append([1.0,0,0,1.0])
        # Create covariance matrix
        Sig = np.reshape(cov, (2,2))
        # Recalculate new parameters
        Mu = np.array([0.0,1.0])
        X, Y, pos = initial_cond(Mu, Sig)
        F = multivariate_normal(Mu, Sig)
        Z = F.pdf(pos)
        # Create covariance matrix
        Sig = np.reshape(cov, (2,2))
    Prior = Z/Z.max()
    # Recalculate new parameters
    ax1.clear()
    ax2.clear()
    
    ax1.plot_surface(X, Y, Z, linewidth=1, antialiased=True,facecolors=cm.viridis(Prior))
    ax1.set_xlabel("X")
    ax1.set_ylabel("Y")
    ax1.set_zlabel("P(X,Y)")
    
    ax2.contourf(X, Y, Z, cmap=cm.viridis)
    ax2.set_xlabel("X")
    ax2.set_ylabel("Y") 
    # Draw both canvases
    canvas1.draw()
    canvas2.draw()
    return



def submit2():
    # Conditional probability boxes
    ax1.clear()
    ax2.clear()

    # type flag: 0 for conditional, 1 for marginal
    type_flag = dirs.get()
    # direction flag: (0) for prior of some known X or marginal independent of x , (1) opposite
    dir_flag = types.get()

    # Copied from submit()
    cov = []
    mu = []
    # Get the values from the text entry boxes
    if (len(entxx.get()) !=0 and len(entxy.get())!=0 and len(entyx.get())!=0 and len(entyy.get())!=0 
       and len(entmux.get()) !=0 and len(entmuy.get()) !=0):
        mu.append(float(entmux.get())) 
        mu.append(float(entmuy.get()))
        cov.append(float(entxx.get()))
        cov.append(float(entxy.get()))
        cov.append(float(entyx.get()))
        cov.append(float(entyy.get()))
        # Create covariance matrix
        Sig = np.reshape(np.array(cov), (2,2))
        # create covariance matrix
        Mu = np.array(mu)
        # Recalculate new parameters
        X, Y, pos = initial_cond(Mu,Sig)
        F = multivariate_normal(Mu, Sig)
        Z = F.pdf(pos)
    else:
        cov.append([1.0,0,0,1.0])
        # Create covariance matrix
        Sig = np.reshape(cov, (2,2))
        # Recalculate new parameters
        Mu = np.array([0.0,1.0])
        X, Y, pos = initial_cond(Mu, Sig)
        F = multivariate_normal(Mu, Sig)
        Z = F.pdf(pos)
        # Create covariance matrix
        Sig = np.reshape(cov, (2,2))
    # end of copy of submit()   

    if (len(entxmin.get())!=0 and len(entxmax.get())!=0 and len(entymin.get())!=0 and len(entymax.get())!=0):
        # Calculate the appropriate location on the axes
        yminent = float(entymin.get())
        ymaxent = float(entymax.get())
        xminent = float(entxmin.get())
        xmaxent = float(entxmax.get())
        # Ensure that the range is visible on the plot
        if (abs(yminent - ymaxent) < 0.2):
            ymaxent =yminent+0.2
        if (abs(xminent - xmaxent)< 0.2):
            xmaxent = xminent+0.2
        # Convert coordinate to index    
        ymin = int((yminent-Y[0,0])/dx)
        xmin = int((xminent-X[0,0])/dx)
        ymax = int((ymaxent-Y[0,0])/dx)
        xmax = int((xmaxent-X[0,0])/dx)
        
        # Normalize the colormap values
        Prior = Z/Z.max()
        if (type_flag == 1):
            Zy, zx = margins(Z)
            Zy = numpy.squeeze(Zy, axis=None)
            Zx = np.transpose(zx[0])
            Ztemp = np.zeros(np.shape(Z))
            if (dir_flag == 1): # Marginal of some X
                Prior[:,99] = 10
                Prior[ymin:ymax,99] = 0.4
                Ztemp[:,99] = Zy[:]
                Prior = Ztemp/Ztemp.max()
            else: # Marginal of some Y
                Prior[99,:] = 10   
                Prior[99,xmin:xmax] = 0.4
                Ztemp[99,:] = Zx
                Prior = Ztemp/Ztemp.max()
            ax1.plot_surface(X, Y, Ztemp, linewidth=1, antialiased=True,facecolors=cm.viridis(Prior))
            ax1.set_xlabel("X")
            ax1.set_ylabel("Y")
            ax1.set_zlabel("P")
            
        elif (type_flag == 0):
            if (dir_flag == 1):
                Prior[:,xmin:xmax] = 0.4
                Prior[ymin:ymax,xmin:xmax] = 10
            elif (dir_flag == 0):
                Prior[ymin:ymax,:] = 0.4                  
                Prior[ymin:ymax,xmin:xmax] = 10
            
            ax1.plot_surface(X, Y, Z, linewidth=1, antialiased=True,facecolors=cm.viridis(Prior))
            ax1.set_xlabel("X")
            ax1.set_ylabel("Y")
            ax1.set_zlabel("P")
            
        if (dir_flag == 1):
            ax2.plot(Y[ymin:ymax,xmin:xmax], Z[ymin:ymax,xmin:xmax])
            ax2.axvspan(ymin,ymax, color='red', alpha=0.5)
            ax2.set_xlabel("Y")
            ax2.set_ylabel("P(Y|X)")
        elif (dir_flag == 0):
            ax2.plot(X[ymin:ymax,:], Z[ymin:ymax,:])
            ax2.axvspan(xmin,xmax, color='red', alpha=0.5)
            ax2.set_xlabel("X")
            ax2.set_ylabel("P(X|Y)")            
    else:
        # Plot the new data
        Prior = Z/Z.max()  
        
        ax1.plot_surface(X, Y, Z, linewidth=1, antialiased=True,facecolors=cm.viridis(Prior))
        ax1.set_xlabel("X")
        ax1.set_ylabel("Y")
        ax1.set_zlabel("P(X,Y)")
        
        ax2.contourf(X, Y, Z, cmap=cm.viridis)
        ax2.set_xlabel("X")
        ax2.set_ylabel("Y") 
    
    # Draw method 
    canvas1.draw()
    canvas2.draw()
    return 
      
    
def initial_cond(mu, Sig):
    """Creat the initial conditions for a bivariate normal here
    This will take the mean vector, mu and a covariance matrix Sig
    Default values have been picked."""
    N =100
    limsx = [(mu[0]-4),(mu[0]+4)]
    limsy = [(mu[1]-4),(mu[1]+4)]
    # Create the axes and mesh grid
    X = np.linspace(limsx[0],limsx[1],N)
    Y = np.linspace(limsy[0],limsy[1],N)
    X,Y = np.meshgrid(X,Y)
    #Pack X and Y into a single 3D array
    pos = np.zeros(X.shape +(2,))
    pos[:,:,0] = X
    pos[:,:,1]= Y
    return X,Y,pos

# Bivariate distribution
Here we will create and visualize a bivariate distribution that is defined in the following way:
\begin{equation*}
f(\overrightarrow{x}) = \frac{1}{\sqrt{(2\pi)^{2} \text{ det}(H) }}exp(-\frac{1}{2}(x-\mu)^T\cdot H^{-1} \cdot (x-\mu))
\end{equation*}

Where $\mu$ is a 2D vector: 
\begin{equation*}
\mathbf{\mu} =  \begin{pmatrix}
\ \mu_{x} \\
\ \mu_{y}
\end{pmatrix}
\end{equation*}

The covariance matrix, H, defined as:
\begin{equation*}
\mathbf{H} = \begin{pmatrix}
\ \sigma_{xx}^{2} & \sigma_{xy}^{2} \\
\ \sigma_{yx}^{2} & \sigma_{yy}^{2}
\end{pmatrix}
\end{equation*}

Notice that $\sigma_{xx}^2$ and $\sigma_{yy}^2$ represent the variance in x and y, and the off-diagonals represent the covariance terms.
## Exercise 1: Visualization of the bivariate distribution
Run the floowing cell to open a GUI that will aid in the visualization of a bivariate ditribution with differing covariance matrix values. 
    

## Create a plot showing the new distribution
We are generating a surface plot with a contour plot below it

In [3]:
root = tk.Tk()
root.wm_title("Bivariate Distribution GUI")

# Initialize the plot
mu = np.array([0.0,1.0])
Sig = np.array([[1.0, -0.5],[-0.5,2.0]])

X, Y, pos = initial_cond(mu,Sig)
dx = X[0,1]-X[0,0]
F = multivariate_normal(mu, Sig)
Z = F.pdf(pos)

# type flag: 0 for conditional, 1 for marginal
types = tk.IntVar()
# direction flag: (0) for prior of some known X or marginal independent of x , (1) opposite
dirs = tk.IntVar()

# Create 2 frames for the GUI: 1 for the Covariance matrix entries and 1 for the two graphs
frame1 = tk.Frame(root)
frame2 = tk.Frame(root)
frame3 = tk.Frame(root)
frame4 = tk.Frame(root)
# Pack the frames 
frame1.grid(column=0,row=0)
frame2.grid(column=1,row=0)
frame3.grid(column=2,row=0)
frame4.grid(column=3,row=0)

# ---------------- Frame 1: Gaussian 3D plot --------------------------------------------------
fig1 = Figure(figsize = (5,4),dpi = 100)
canvas1 = FigureCanvasTkAgg(fig1, master = frame1) # Tk drawing area
canvas1.draw()
ax1 = fig1.add_subplot(111,projection = "3d")
ax1.plot_surface(X, Y, Z, rstride=3, cstride=3, linewidth=1, antialiased=True,cmap=cm.viridis)
ax1.set_xlabel("X")
ax1.set_ylabel("Y")
ax1.set_zlabel("P(X,Y)")


toolbar1 = NavigationToolbar2Tk(canvas1, frame1)
toolbar1.update()
canvas1.get_tk_widget().pack(side=tk.LEFT, fill=tk.BOTH, expand=1)


# ---------------- Frame 2: Contour plot -------------------------------------------------------
fig2 = Figure(figsize = (5,4),dpi = 100)
canvas2 = FigureCanvasTkAgg(fig2,master = frame2)
canvas2.draw()
ax2 = fig2.add_subplot(111)
ax2.contourf(X, Y, Z, cmap=cm.viridis)
ax2.set_xlabel("X")
ax2.set_ylabel("Y")

toolbar2 = NavigationToolbar2Tk(canvas2, frame2)
toolbar2.update()
canvas2.get_tk_widget().pack(side=tk.LEFT, fill=tk.BOTH, expand=1)


# ---------- Frame 3 and 4: covariance, mu, and data range entry boxes/labels -------------------------------
# MU setting label
label_mu = tk.Label(frame3, text='Set \u03bc vector:',font=('calibre', 14, 'bold'))
label_blank1 = tk.Label(frame4, text=' ',font=('calibre', 14, 'bold'))

# Label and entry mu x:
labelmux = tk.Label(frame3, text = "\u03bcx", font=('calibre', 12, 'bold'))
entmux= tk.Entry(frame3)

# Label and entry mu y:
labelmuy = tk.Label(frame4, text = "\u03bcy", font=('calibre', 12, 'bold'))
entmuy= tk.Entry(frame4)

# COV setting label
label_cov = tk.Label(frame3, text='Set \u03c3 matrix:',font=('calibre', 14, 'bold'))
label_blank2 = tk.Label(frame4, text=' ',font=('calibre', 14, 'bold'))

# Label and entry cov xx:
labelxx = tk.Label(frame3, text = "\u03c3xx", font=('calibre', 12, 'bold'))
entxx= tk.Entry(frame3)

# Label and entry cov xy:
labelxy = tk.Label(frame4, text = "\u03c3yx", font=('calibre', 12, 'bold'))
entxy= tk.Entry(frame4)

# Label and entry cov yx:
labelyx = tk.Label(frame3, text = "\u03c3yx", font=('calibre', 12, 'bold'))
entyx= tk.Entry(frame3)

# Label and entry cov yy:
labelyy = tk.Label(frame4, text = "\u03c3yy", font=('calibre', 12, 'bold'))
entyy= tk.Entry(frame4)

# Covariance and mu entry button
button1 = tk.Button(frame3,text="Enter",command=submit)
label_blank3= tk.Label(frame4, text=' ',font=('calibre', 12, 'bold'))


# Entry boxes for data x-range
labelxmin = tk.Label(frame3, text = "xmin", font=('calibre', 12, 'bold'))
entxmin= tk.Entry(frame3)

labelxmax = tk.Label(frame4, text = "xmax", font=('calibre', 12, 'bold'))
entxmax= tk.Entry(frame4)

# Entry boxes for data y-range
labelymin = tk.Label(frame3, text = "ymin", font=('calibre', 12, 'bold'))
entymin= tk.Entry(frame3)

labelymax = tk.Label(frame4, text = "ymax", font=('calibre', 12, 'bold'))
entymax= tk.Entry(frame4)
# Radiobutton for marginal and conditional probability experiments
marg_button = tk.Radiobutton(frame3, text="Marginal", variable=types,
                            indicatoron=True, value=1)
cond_button = tk.Radiobutton(frame3, text="Conditional", variable=types,
                            indicatoron=True, value=0)
dirX_button = tk.Radiobutton(frame4, text="P(X|Y) or P(Y)", variable=dirs,
                            indicatoron=True, value=0)
dirY_button = tk.Radiobutton(frame4, text="P(Y|X) or P(X)", variable=dirs,
                            indicatoron=True, value=1)

button2 = tk.Button(frame3,text="Enter",command=submit2)

# Pack labels, entries, and Covariance button:
label_mu.pack(fill="x")
label_blank1.pack(fill="x")

labelmux.pack()
entmux.pack()

labelmuy.pack()
entmuy.pack()

label_cov.pack(fill="x")
label_blank2.pack(fill="x")

labelxx.pack()
entxx.pack()

labelxy.pack()
entxy.pack()

labelyx.pack()
entyx.pack()

labelyy.pack()
entyy.pack()

button1.pack(fill="x")
label_blank3.pack(fill='x')

labelxmin.pack()
entxmin.pack()

labelxmax.pack()
entxmax.pack()

labelymin.pack()
entymin.pack()

labelymax.pack()
entymax.pack()


marg_button.pack()
cond_button.pack()

dirX_button.pack()
dirY_button.pack()

button2.pack(fill="x")

tk.mainloop()

In [49]:
m1, m2=margins(Y[1:10,1:10])

In [67]:
m1.shape

(9, 1)

array([-24.81818182])